# Grover's search algorithm

Grover's search algorithm gives an averge O(sqrt) order improvement over classical methods, to invert a black-box function. It is often said that Grover's search algorithm can be used to search an "unstructured database," like a lookup table. However, this requires a way to read a lookup table into quantum form that the algorithm can operate on, which is not a given, in the noisy-intermediate-scale-quantum ("NISQ") era. At least, we can imagine a unitary way to do this, if any hardware associated with any unitary operation is attainable.

We search a lookup table with Grover's algorithm. Our lookup table is 1 "page" of _classical data_. To read  the lookup table into quantum registers, we assume the availability of quantum "indexed addressing" variants of "load," "add-with-carry," and "subtract-with-carry" operations. An "index register" of qubits points to an offset into the classical value page, to load a value within the page into a "value register." The qubit-based index register can represent an index in **superposition**, and we can load values in superposition of the index pointer offset. This hypothetical operation is at least unitary, and reversible. This might be hypothetically attainable with a superconducting classical RAM cache.

First, we define useful constants, representing the lengths of the index and value registers, the index of a carry bit for addition and subtraction, and the value we're looking for in the lookup table.

In [1]:
# Number of qubits in index register
indexLength = 8
# Number of qubits in value register
valueLength = 8
# Carry flag qubit index
carryIndex = indexLength + valueLength
# Value we're searching for
TARGET_VALUE = 100

**We don't know the key associated with the target value.** This is what we're searching for. However, in this example, we initialize our lookup table with knowledge of this value. We can hand the initialized lookup table to a subroutine to perform Grover's search, afterward, with the point being that we don't need to tell the subroutine the value of this key.

In [2]:
# (We don't "know" this, for purpose of the problem.)
TARGET_KEY = 230

We initialize a set of quantum registers that are large enough for the search algorithm.

In [3]:
from pyqrack import QrackSimulator
sim = QrackSimulator(indexLength + valueLength + 1)

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_Gen9_HD_Graphics_NEO.ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir


We initialize the lookup table to search, but this might be handed pre-initialized to our Grover's algorithm subroutine, without knowledge of its contents.

In [4]:
toLoad = [1] * (1 << indexLength)
toLoad[TARGET_KEY] = TARGET_VALUE

We define the qubit indices of index and value registers.

In [5]:
indexQubits = [i for i in range(indexLength)]
valueQubits = [(v + indexLength) for v in range(valueLength)]

We start the **index register** in **maximal superposition**, pointing with equal probability to every element in the lookup table page.

In [6]:
for i in indexQubits:
    sim.h(i)

This is the "magic" operation we don't necessarily have in NISQ hardware, yet: we load from a classical page of data with a superposed quantum offset index pointer register.

In [7]:
sim.lda(indexQubits, valueQubits, toLoad)

This is a closed form expression for the ideal number of iterations for Grover's search, if there is exactly one match to our search target in the lookup table. For a page of 256 values, (pointed to by the index register permutation capacity of 256,) this happens to be 12 iterations.

In [8]:
import math
optIter = math.floor(math.pi / (4 * math.asin(1 / math.sqrt(1 << indexLength))))

This method is commonly called our "oracle." All it does is "tag" a specific value with a phase factor of "pi".

In [9]:
def oracle(valueQubits, TARGET_VALUE, sim):
    sim.sub(TARGET_VALUE, valueQubits)
    sim.macmtrx(valueQubits[1:], [-1, 0, 0, 1], valueQubits[0])
    sim.add(TARGET_VALUE, valueQubits)

This is our Grover's search iteration loop. (Grover's search is a form of "amplitude amplification" algorithm.)

In [10]:
for i in range(optIter):
    # Tag the search target VALUE with a full phase reversal.
    oracle(valueQubits, TARGET_VALUE, sim)

    # Invert the original initialization fully, to map back to a |0> permutation state, except for our phase effects.
    sim.x(carryIndex)
    sim.sbc(carryIndex, indexQubits, valueQubits, toLoad)
    sim.x(carryIndex)
    for j in indexQubits:
        sim.h(j)

    # Perform a full phase reversal on the |0> permutation state.
    sim.macmtrx(indexQubits[1:], [-1, 0, 0, 1], indexQubits[0])

    # Re-initialize as if preparing for the first iteration of Grover's algorithm.
    for j in indexQubits:
        sim.h(j)
    # We could "reverse sign of global phase," here, but that cannot be measured.
    sim.adc(carryIndex, indexQubits, valueQubits, toLoad)

Grover's search has a probabilistic chance of success or failure, but the chance of success is high enough, in this case, that we should match 10 out of 10 attempts, (almost every time).

In [11]:
print(sim.measure_shots(valueQubits, 10))
print(sim.measure_shots(indexQubits, 10))

[100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
[230, 230, 230, 230, 230, 230, 230, 230, 230, 230]


Our target **value** in the lookup table was **100**, and it corresponds to a lookup table **key** of **230**.